In [2]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
my_date_handler = lambda year, month, day, **kwargs: "%04i-%02i-%02i" % (year, month, day)
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

ProgrammingError: Oracle Client library has already been initialized

In [3]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

### Branch Shipping
- 조건
    1. waiting_for_out에서만 출고가능
    2. 선택된 Cell이 있어야만 출고가능
        - 선택된 Cell 없으면 GOOD상태인 모든 대리점 진행
    3. 선택된 Cell에서 대리점이 2개이상이면 출고 불가 같은 대리점이라도 지점다르면 다르게 출고
    4. 인수증방식 선택되어야 출고가능
    
- 배송방법
    1. 일반배송, 무진동차량
        - 기존 프로세스로 진행
    2. 택배

In [4]:
def row_nm_check(xw_book_name=xw.books[0]):
    """
    Return Dict, get activated sheet's name(str), get selected row's number(list)

    each_list는 연속된 row번호들을 전부 계산하여 리스트안에 전부 각각 위치하도록 한다.
    each_list의 모든 값은 int
    """
    ## SpecialCells(12) 셀이 한개만 클릭됬을 경우에는 제대로 작동 불가

    # 셀한개만 클릭할경우 $의 개수가 2개이고 2개이상일경우 $의 개수는 4개 이다.
    sel_cell = xw_book_name

    count_dollar = sel_cell.selection.address.count("$")
    
    # 셀한개만 클릭할경우
    if count_dollar == 2 :
        
        sel_rng = [sel_cell.selection.address]
    else : 
    
        sel_rng = xw_book_name.selection.api.SpecialCells(12).Address.split(",")
    
    range_list = []
    
    for rng in sel_rng:

        if ":" in rng:
            num_0 = rng.split(":")[0].split("$")[-1]
            num_1 = rng.split(":")[1].split("$")[-1]
            #범위선택이 한 컬럼이 아닌 여러 컬럼을 동시에 선택된 경우
            if num_0 == num_1 :
                range_list.append(num_0)
            else : 
                range_list.append(str(num_0) + ',' + str(num_1))
        else :
            range_list.append(rng.split("$")[-1])

    fin_list = []
    for rng_val in range_list:
        if ',' in rng_val:
            tmp_split = rng_val.split(',')
            cnts = int(tmp_split[1])-int(tmp_split[0])
            tmp_list = []
            for i in range(cnts+1):
                tmp_list.append(int(tmp_split[0])+i)
                
            fin_list = fin_list + tmp_list
        else :
            fin_list.append(int(rng_val))
            
    dict_row_num_sheet_name = {"sheet_name":xw_book_name.selection.sheet.name,'selection_row_nm':range_list,'each_list':fin_list}
    
    
    return dict_row_num_sheet_name

In [5]:
def get_row_list_to_string(seleted_row_list) :
    return ' '.join(seleted_row_list).replace(',','~').replace(' ', ', ')

In [34]:
def get_out_table_for_branch(sheet_name=wb_cy.selection.sheet,index_row_number=9):
    """
    xlwings.main.Sheet를 인수로 입력, 해당시트의 index행번호 default = 9 (int)
    list형태의 출고하는 시트의 index들을 반환한다.
    """
    out_row_nums = get_row_list_to_string(row_nm_check(wb_cy)['selection_row_nm'])
    last_col = sheet_name.range("XFD9").end('left').column
    idx_row_num = index_row_number
    col_names = sheet_name.range(sheet_name.range(int(idx_row_num),1),sheet_name.range(int(idx_row_num),last_col)).value
    
    for idx ,i in enumerate(col_names):
        
        if i == None:
            continue
        elif '_INDEX' in i :
            col_num = idx
    
    df_so = pd.DataFrame()
    
    # wb_cy.app.alert(out_row_nums)
    try :
        row_list = out_row_nums.replace(' ','').split(',')
    except:
        row_list = [out_row_nums]
    for row in row_list :
        
        # 연속된 행인 경우
        if '~' in str(row) :
            left_row =int(row.split('~')[0])
            right_row = int(row.split('~')[1])
            rng = sheet_name.range(sheet_name.range(left_row,1),sheet_name.range(right_row,last_col))
            df_so = pd.concat([df_so,pd.DataFrame(sheet_name.range(rng).options(numbers=int).value)]) 
        else :
            
            left_row =int(row)
            right_row = int(row)
            rng = sheet_name.range(sheet_name.range(left_row,1),sheet_name.range(right_row,last_col))
            df_so = pd.concat([df_so,pd.DataFrame(sheet_name.range(rng).options(numbers=int).value).T])

    
    return list(df_so[col_num])

In [38]:
get_out_table_for_branch()

[20470]

### Branch Shipping
- 조건
    1. waiting_for_out에서만 출고가능
    2. 선택된 Cell이 있어야만 출고가능
        - 선택된 Cell 없으면 GOOD상태인 모든 대리점 진행
    3. 선택된 Cell에서 대리점이 2개이상이면 출고 불가 같은 대리점이라도 지점다르면 다르게 출고
    4. 인수증방식 선택되어야 출고가능
    
- 배송방법
    1. 일반배송, 무진동차량
        - 기존 프로세스로 진행
    2. 택배

In [114]:
def branch_ship_ready():


    sel_sht = wb_cy.selection.sheet
    mode = sel_sht.range("H4")
    in_date = sel_sht.range("P4")
    in_date.clear_contents()
    if mode.value != 'waiting_for_out':
        wb_cy.app.alert("'waiting_for_out' 모드에서만 사용 가능합니다.","Quit")
        return

    col_names = list(pd.DataFrame(cursor.execute(f"select column_name from user_tab_columns where table_name = upper('SHIPMENT_INFORMATION')").fetchall())[0])

    val_rng = sel_sht.range("P3:P7")
    val_rng.clear_contents()

    idx_list = get_out_table_for_branch()
    idx_list_str = list(map(lambda e : str(e),idx_list))
    
    db_row_list =[]
    try:
        for idx in idx_list:
            qry = f"select * from SHIPMENT_INFORMATION where si_index = '{idx}'"
            db_row_list.append(cursor.execute(qry).fetchone())
        df_br = pd.DataFrame(db_row_list,columns=col_names)
    except:
        wb_cy.app.alert("시트 내용이 손상됬거나 값이 있는 테이블이외의 셀을 선택하셨습니다. 정상적인 선택인 경우 BringDATA 버튼 클릭후 다시 시도해주세요.","Quit")
        return
    
    # 대리점 품목인지여부
    is_branch = len(df_br)==len(df_br[df_br['REMARK']=='대리점'])
    if is_branch != True:
        wb_cy.app.alert("대리점이 아닌 품목이 선택되었습니다. SHIP TO 컬럼의 대리점명이 오타일 경우 ChangeCell기능을 이용하여 정확한 이름으로 변경 및 REMARK컬럼은 '대리점'으로 변경 후 다시시도해주세요","Quit")
        return
    # 출고가능 여부
    is_good = len(df_br)==len(df_br[df_br['STATE'].str.contains("GOOD")])
    if is_good != True:
        wb_cy.app.alert("선택하신 품목의 STATE가 GOOD_WR 또는 GOOD_ESLE 상태에서만 진행 가능합니다. HOLDING일경우 검수완료를 진행 후 다시 시도해주세요.","Quit")
        return
    # ship_to 종류
    selected_site_count = len(set(df_br['SHIP_TO']))
    if selected_site_count != 1:
        wb_cy.app.alert("서로 다른 대리점들이 선택 되었거나 SHIP_TO에 오타가 있습니다. 확인해주세요","Quit")
        return
    branch_name = list(set(df_br['SHIP_TO']))[0]
    
    val_rng[0].value = ",".join(idx_list_str) # 출고행번호
    val_rng[2].value = branch_name # 대리점명

In [120]:
branch_ship_ready()

In [336]:
def branch_out():
    sel_sht = wb_cy.selection.sheet
    mode = sel_sht.range("H4")
    val_rng = sel_sht.range("P3:P7")
    if mode.value != 'waiting_for_out':
        wb_cy.app.alert("'waiting_for_out' 모드에서만 사용 가능합니다.","Quit")
        return
    col_names = list(pd.DataFrame(cursor.execute(f"select column_name from user_tab_columns where table_name = upper('SHIPMENT_INFORMATION')").fetchall())[0])
    input_values = val_rng.options(numbers=int, dates=my_date_handler).value
    del_method = input_values[3]
    
    if input_values.count(None) > 0 :
        wb_cy.app.alert("ONLY FOR BRANCH의 FORM에 빈 값이 있습니다. 입력 후 시도 해주세요!","Quit")
        return
    idx_list = input_values[0].split(',')
    db_row_list=[]
    try:
        for idx in idx_list:
            qry = f"select * from SHIPMENT_INFORMATION where si_index = '{idx}'"
            db_row_list.append(cursor.execute(qry).fetchone())
        df_br = pd.DataFrame(db_row_list,columns=col_names)
    except:
        wb_cy.app.alert("시트 내용이 손상됬거나 값이 있는 테이블이외의 셀을 선택하셨습니다. 정상적인 선택인 경우 BringDATA 버튼 클릭후 다시 시도해주세요.","Quit")
        return
    parcel_list = list(set(df_br['PARCELS_NO']))
    
    if input_values[3] =='택배':
        while True:
            courier_num_list = input_delivery_invoice_number_for_branch(parcel_list)
            pacels_dict = dict(zip(parcel_list,courier_num_list))
            alert_str = str(pacels_dict).replace(",","\n").replace('{','').replace('}',"")
            answer = wb_cy.app.alert("당신의 입력 : \n"+alert_str+" \n 수정하시겠습니까?","CONFIRM",buttons="yes_no_cancel")
            if answer == 'no':
                del_method = pacels_dict
                break
            if answer == 'cancel':
                wb_cy.app.alert("종료합니다.","Quit")
                return
            
    # 1. update_shipdate 매서드사용으로 DB변화완료
    ShipmentInformation.update_shipdate_for_branch(input_values[0],input_values[1],del_method=del_method)
    
    # 2. sht_proctect() 매서드로 edit_mode진입
    sht_protect(False)
    bring_data_from_db()
    sht_protect(True)
    
    # 3. mail form작성
    ws_br_out= xw.Book('print_form.xlsx').sheets['BRANCH_SHIPPING']
    carton_count = len(df_br[pd.notna(df_br['NM_OF_PACKAGE'])])
    branch_out_col_names = [
        'ARRIVAL_DATE',
        'AWB_NO',
        'TRIP_NO',
        'NM_OF_PACKAGE',
        'PARCELS_NO',
        'ORDER_NM',
        'SHIP_TO'
    ]
    df_br_out = df_br[branch_out_col_names]
    df_br_out['COURIER_INVOICE_NM'] = None

    last_row = ws_br_out.range("A1048576").end('up').row
    if last_row < 9 :
        last_row = 8
    req_rng = ws_br_out.range((9,1),(last_row,"H"))
    req_rng.api.Borders.LineStyle = -4142
    req_rng.clear_contents()

    ws_br_out.range("A7").value = f"총 {carton_count} CARTON 입니다."
    if input_values[3] == "택배":
        ws_br_out.range("D7").value = f"{input_values[1]}에 택배로 출고 완료 되었습니다."
        for k,y in pacels_dict.items():
            df_idx = df_br_out[df_br_out['PARCELS_NO']==k].index
            df_br_out['COURIER_INVOICE_NM'].loc[df_idx] = y
    else : 
        ws_br_out.range("D7").value = f"{input_values[1]}에 {input_values[3]}(으)로 배송 예정입니다."
        df_br_out['COURIER_INVOICE_NM'] = input_values[3]

    df_br_out.set_index("ARRIVAL_DATE",inplace=True)
    ws_br_out.range("A8").value = df_br_out

    last_row = ws_br_out.range("A1048576").end('up').row
    if last_row < 9 :
        last_row = 8
    req_rng = ws_br_out.range((8,1),(last_row,"H"))
    req_rng.autofit()
    req_rng.api.Borders.LineStyle = 1
    
    # 메일 작성 및 저장
    outlook_send=cli.Dispatch("Outlook.Application")
    branch_db = cursor.execute('select * from branch').fetchall()
    branch_db_df = pd.DataFrame(branch_db)
    br_row_df =branch_db_df[branch_db_df[0]==input_values[2]]

    to_mail = br_row_df.iloc[0][2]
    cc_mail = br_row_df.iloc[0][2]

    # 메일 저장 작업
    file_name = 'BR_HTML_CONTENT.html'
    br_html = os.getcwd() + '\\'+file_name
    ws_br_out.to_html(file_name)
    html_body = html.unescape(open(br_html).read())
    mail_obj = outlook_send.CreateItem(0)
    mail_obj.To = to_mail
    mail_obj.CC = cc_mail
    mail_obj.Subject = input_values[2] + " " + input_values[1] + " 배송 정보"
    mail_obj.HTMLBody = html_body
    os.remove(br_html)
    mail_obj.Save()

In [347]:
for val in idx_list:
    print(f"UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = {val}")

UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = 20477
UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = 20478
UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = 20479
UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = 20480
UPDATE SHIPMENT_INFORMATION SET COMMENT = '일반배송' WHERE SI_INDEX = 20481


In [ ]:
# 1. update_shipdate 매서드사용으로 DB변화완료
# branch shipdate 업데이트의 경우 comment에 배송방법 및 송장번호 적을 것
# 2. sht_proctect() 매서드로 edit_mode진입
# 3. mail form작성

In [301]:
import win32com.client as cli
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩

In [308]:
list(pd.DataFrame(branch_list)[0])

['동일과학',
 '이효바이오사이언스',
 '서린바이오사이언스',
 '비앤비대전',
 '바이오토프',
 '유비코리아',
 '바이나리',
 '지더블유바이텍가산',
 '지더블유바이텍화성',
 '닥터바이오',
 '지더블유바이텍광주']

In [330]:
ws_br_out

<Sheet [print_form.xlsx]BRANCH_SHIPPING>

In [ ]:
def update_shipdate_for_branch(self,get_each_index_num,ship_date,status='SHIP_CONFIRM',del_method='일반배송'):
    """
    so out시 db의 ship_date수정 및 pod완료시 pod_date 수정이 필요하다.
    # ws_si 시트의 해당하는 db 값 수정 대리점 출고전용
    """
    cur = self.DataWarehouse_DB
    if type(get_each_index_num) is list:
        idx_list = get_each_index_num
    elif type(get_each_index_num) is dict:
        idx_list=get_each_index_num['idx_list']
    else:
        idx_list = [get_each_index_num]
    
    # comment 수정
    if type(del_method) == dict :
        pacels_dict = del_method
        for key in pacels_dict.keys():
            query = f"UPDATE SHIPMENT_INFORMATION SET COMMENT = '{pacels_dict[key]}' WHERE PARCELS_NO = {key}"
    else : 
        for val in idx_list:
            query = f"UPDATE SHIPMENT_INFORMATION SET COMMENT = '{del_method}' WHERE SI_INDEX = {val}"

    for val in idx_list:
        # ship_date
        query = f"UPDATE SHIPMENT_INFORMATION SET SHIP_DATE = '{ship_date}' WHERE SI_INDEX = {val}"
        cur.execute(query)
        # timeline
        bring_tl_query = f"SELECT TIMELINE FROM SHIPMENT_INFORMATION WHERE SI_INDEX = {val}"
        try:
            json_tl = cur.execute(bring_tl_query).fetchone()[0]
        except:
            self.WB_CY.app.alert(f" SI_INDEX : {val} 해당 INDEX는 DB에 등록되지 않은 INDEX입니다. Data는 DataInput기능으로만 저장 가능합니다. 종료합니다.","Quit")
            return 
        cur.execute(bring_tl_query).fetchone()[0]
        timeline_query = f"UPDATE SHIPMENT_INFORMATION SET TIMELINE = '{create_db_timeline(json_tl)}' WHERE SI_INDEX = {val}"
        cur.execute(timeline_query)
        # state
        update_state_query = f"UPDATE SHIPMENT_INFORMATION SET STATE = '{status}' WHERE SI_INDEX = {val}"
        cur.execute(update_state_query)

1

In [198]:
def input_delivery_invoice_number_for_branch(parcel_list):
    wb_cy.app.alert("Parcel_NO에 맞는 송장번호를 입력해주세요.","INFO")
    courier_num_list = []
    for i in range(len(parcel_list)):
        courier_num_list.append(wb_cy.app.api.InputBox(f"'{parcel_list[i]}'에 맞는 송장번호를 입력해주세요.","Delivery invoice number INPUT",Type=2))
    return courier_num_list